<a href="https://colab.research.google.com/gist/Sayam753/f434492fc19f78bb93f3002cdecfd002/distributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distributions Enhancement Proposal - PyMC4

## Add transformations to bounded distributions

At the current state of PyMC4, only `Log` and `Logit` transformations are included to do `mcmc` sampling or `vi` approximations. No transformations have been there for Bounded distributions. This notebook explores some ways of adding `Interval` transform.

In [ ]:
!git checkout master

Switched to branch 'master'
Your branch is up to date with 'origin/master'.


In [ ]:
import arviz as az
import math
import numpy as np
import pymc4
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions
tfb = tfp.bijectors

In [ ]:
experiments = np.array([1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

@pymc4.model
def model():
    prob = yield pymc4.Uniform('p', 0., 1.)
    ll = yield pymc4.Bernoulli('ll', prob, observed=experiments)

In [ ]:
advi = pymc4.fit(model())

Instructions for updating:
The signature for `trace_fn`s passed to `minimize` has changed. Trace functions now take a single `traceable_quantities` argument, which is a `tfp.math.MinimizeTraceableQuantities` namedtuple containing `traceable_quantities.loss`, `traceable_quantities.gradients`, etc. Please update your `trace_fn` definition.


In [ ]:
np.sum(np.isnan(advi.losses))  # there seems are a large section of ELBO loss containing nans

6867

One can also pass `validate_args=True` to each distribution while doing VI, and justify support matching constraint failing.

## Possible Solutions
## 1. Using conditionals

### Transformations are defined as -

```python
from pymc4.distributions.transforms import JacobianPreference, Transform

class Interval(Transform):
    name = "interval"
    JacobianPreference = JacobianPreference.Backward

    def __init__(self, lower_limit, upper_limit):
        if math.isinf(lower_limit) and math.isinf(upper_limit):
            transform = tfb.Identity()
            
        elif math.isinf(lower_limit) and math.isfinite(upper_limit):
            transform = tfb.Chain([tfb.Shift(upper_limit), tfb.Scale(-1), tfb.Exp()])  # upper - exp(x)
            
        elif math.isfinite(lower_limit) and math.isinf(upper_limit):
            transform = tfb.Chain([tfb.Shift(lower_limit), tfb.Exp()])  # exp(x) + lower
        
        else:
            transform = tfb.Sigmoid(low=lower_limit, high=upper_limit)  # interval
        self._transform = transform

    def forward(self, x):
        return self._transform.inverse(x)

    def inverse(self, z):
        return self._transform.forward(z)

    def forward_log_det_jacobian(self, x):
        return self._transform.inverse_log_det_jacobian(x, self._transform.inverse_min_event_ndims)

    def inverse_log_det_jacobian(self, z):
        return self._transform.forward_log_det_jacobian(z, self._transform.forward_min_event_ndims)
```

### Fitting the model -

```python
advi = pymc4.fit(model())

# Traceback
~/Desktop/pymc/pymc4/pymc4/distributions/distribution.py in _init_transform(self, transform)
    280     def _init_transform(self, transform):
    281         if transform is None:
--> 282             return transforms.Interval(self.lower_limit(), self.upper_limit())
    283         else:
    284             return transform

~/Desktop/pymc/pymc4/pymc4/distributions/transforms.py in __init__(self, lower_limit, upper_limit)
    160 
    161     def __init__(self, lower_limit, upper_limit):
--> 162         if math.isinf(lower_limit) and math.isinf(upper_limit):
    163             transform = tfb.Identity()
    164 

TypeError: must be real number, not Tensor

```



## 2. Using tf.cond

### Transformations are defined as -

```python
from pymc4.distributions.transforms import JacobianPreference, Transform

class Interval(Transform):
    name = "interval"
    JacobianPreference = JacobianPreference.Backward

    def __init__(self, lower_limit, upper_limit):
        transform = tf.cond(
            tf.math.is_inf(lower_limit),
            lambda: tf.cond(
                tf.math.is_inf(upper_limit),
                lambda: tfb.Identity(),
                lambda: tfb.Chain([tfb.Shift(upper_limit), tfb.Scale(-1), tfb.Exp()]),  # upper - exp(x)
            ),
            lambda: tf.cond(
                tf.math.is_inf(upper_limit),
                lambda: tfb.Chain([tfb.Shift(lower_limit), tfb.Exp()]),  # exp(x) + lower
                lambda: tfb.Sigmoid(low=lower_limit, high=upper_limit),  # interval
            ),
        )
        self._transform = transform

    def forward(self, x):
        return self._transform.inverse(x)

    def inverse(self, z):
        return self._transform.forward(z)

    def forward_log_det_jacobian(self, x):
        return self._transform.inverse_log_det_jacobian(x, self._transform.inverse_min_event_ndims)

    def inverse_log_det_jacobian(self, z):
        return self._transform.forward_log_det_jacobian(z, self._transform.forward_min_event_ndims)
```

### Fitting the model -

```python
advi = pymc4.fit(model())

# Traceback
/usr/local/lib/python3.7/site-packages/tensorflow/python/util/nest.py in <listcomp>(.0)
    635 
    636   return pack_sequence_as(
--> 637       structure[0], [func(*x) for x in entries],
    638       expand_composites=expand_composites)
    639 

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/func_graph.py in convert(x)
    946               "must return zero or more Tensors; in compilation of %s, found "
    947               "return value of type %s, which is not a Tensor." %
--> 948               (str(python_func), type(x)))
    949       if add_control_dependencies:
    950         x = deps_ctx.mark_as_return(x)

TypeError: To be compatible with tf.eager.defun, Python functions must return zero or more Tensors; in compilation of <function Interval.__init__.<locals>.<lambda>.<locals>.<lambda> at 0x137ceeef0>, found return value of type <class 'tensorflow_probability.python.bijectors.identity.Identity'>, which is not a Tensor.
```

## 3. Manually inspect the distributions and add transformations
This manual addition has already been included in the PR [#289](https://github.com/pymc-devs/pymc4/pull/289) Add Full Rank Approximation to PyMC4. ADVI works but this process does not seem very natural.

## 4. Proposed Approach -
It is difficult to propose a generic solution as we do not know which values to transform beforehand. To solve this, classify the Bounded distributions as -
1. Upper Bounded Distributions
2. Lower Bounded Distributions
3. Interval Bounded Distributions

After adding transformations to these base classes, inherit them to their corresponding distributions. This approach has been proposed after discussions with Maxim.

This proposed implementation is ready to test on a separate branch [https://github.com/Sayam753/pymc4/tree/proposed_transformations](https://github.com/Sayam753/pymc4/tree/proposed_transformations). Any feedback over this.

Also, this discussion in google groups (https://groups.google.com/a/tensorflow.org/forum/#!topic/tfprobability/us6ZzR_WTZU) mentions about automatically applying bijectors in future from TFP side.